In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import wandb
import pickle
from tqdm import tqdm
import nlp_utils as nu
import matplotlib.pyplot as plt
import string
print("PyTorch version:", torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
wandb.login()
train_data_path = '../data/Subtask_1_train.json'
test_data_path = '../data/Subtask_1_test.json'
pickle_save_path = '../data/pickle/'
model_save_path = '../models/'
dataset = pd.read_json(train_data_path)
# train_df
dataset
conversations_df = pd.read_json(train_data_path)['conversation']
conversations_df[0]
unique_labels = set()
unique_speakers = set()

for conv in conversations_df:
    for utterance in conv:
        unique_labels.add(utterance['emotion'])
        unique_speakers.add(utterance['speaker'])
unique_labels
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(list(unique_labels))

label_dict_save_path = pickle_save_path + "label_dict.pkl"
labels_dict = {x:label_encoder.transform([x])[0] for x in label_encoder.classes_}
pickle.dump(labels_dict, open(label_dict_save_path, "wb"))
all_conversations = []

for conv in conversations_df:
    dialog = []
    for sentence in conv:
        speaker = sentence['speaker']
        utterance = nu.preprocess_text(sentence['text'])
        dialog.append(utterance)
    all_conversations.append(dialog)
start_token = '<CLS>'
sep_token = '<SEP>'
"".find
dataset
all_conversations
def find_subsentence_indices(full_sentence, sub_sentence):
    start_index = full_sentence.find(sub_sentence)
    end_index = start_index + len(sub_sentence) - 1
    print(start_index, end_index, full_sentence[start_index:end_index+1])
    return start_index, end_index

def finding(dataset):
    # full_liso = []
    liso = []
    for i in dataset.values:
        id = i[0]
        emo_cau_pairs =  i[2]
        convo = i[1]
        liso.append({"convo" : id,
                     "pairs" : []})
        for j in emo_cau_pairs:
            a1 = j[0]
            a2 = j[1]
            checker1 = a1.split("_")[0]
            checker2 = a2.split("_")[0]
            num1 = int(checker1)
            num2 = int(checker2)
            s1 = a1[len(checker1)+1:]
            s2 = a2[len(checker2)+1:]
            for kk in convo:
                if(kk["utterance_ID"])==num2:
                    # print(kk['text'], s2)
                    a,b = find_subsentence_indices(nu.preprocess_text(kk["text"]), nu.preprocess_text(s2))
                    
                    liso[len(liso)-1]["pairs"].append([num1, num2, a, b])

    return liso

liso=finding(dataset)
liso[529]
all_conversations[0][0][85:113]
all_conversations[529][0][36:74+100]
liso
cause_labels = []
for conv in liso:
    map = {}
    for arr in conv['pairs']:
        if not map.get(arr[0], None):
            map[arr[0]] = []
        map[arr[0]].append(arr[1])
    cause_labels.append(map)
def is_cause(target, utt, conv_id):
    if utt in cause_labels[conv_id].get(target, []):
        return 1
    return 0
all_tc_pairs = []
all_tc_labels = []

for conv_id, conv in enumerate(all_conversations):
    for target_id, target in enumerate(conv):
        for utt_id, utt in enumerate(conv):
            all_tc_pairs.append([target, utt])
            all_tc_labels.append(is_cause(target_id+1, utt_id+1, conv_id))

all_tc_pairs = np.array([np.array(x) for x in all_tc_pairs], dtype='object')
all_tc_labels = np.array(all_tc_labels)

all_tc_pairs
print(all_tc_labels[:20])
train_tc_pairs, dev_tc_pairs, train_tc_labels, dev_tc_labels = train_test_split(all_tc_pairs, all_tc_labels, test_size=0.2, random_state=42)
# train_conversations = []
# dev_conversations = []

# train_utterances = []
# train_emotions = []
# train_speakers = []

# dev_utterances = []
# dev_emotions = []
# dev_speakers = []

# for conv in train_conv_df:
#     dialog = []
#     dialog_emotions = []
#     for sentence in conv:
#         speaker = sentence['speaker']
#         utterance = f'{speaker}: ' + nu.preprocess_text(sentence['text'])
#         emotion = sentence['emotion']
#         train_utterances.append(utterance)
#         dialog.append(utterance)
#         train_speakers.append(speaker)
#         train_emotions.append(sentence['emotion'])
#         dialog_emotions.append(sentence['emotion'])
#     train_conversations.append(dialog)

# for conv in dev_conv_df:
#     dialog = []
#     dialog_emotions = []
#     for sentence in conv:
#         speaker = sentence['speaker']
#         utterance = f'{speaker}: ' + nu.preprocess_text(sentence['text'])
#         emotion = sentence['emotion']
#         dev_utterances.append(utterance)
#         dialog.append(utterance)
#         dev_speakers.append(speaker)
#         dev_emotions.append(sentence['emotion'])
#         dialog_emotions.append(sentence['emotion'])
#     dev_conversations.append(dialog)

# train_emotions = label_encoder.transform(train_emotions)
# dev_emotions = label_encoder.transform(dev_emotions)
def encode_plus(tc_pairs, tokenizer, max_length=256):
    input_ids = []
    attention_masks = []

    for pair in tqdm(tc_pairs, total=len(tc_pairs), desc="Encoding utterances"):
        encoded = tokenizer.encode_plus(pair[0], pair[1],
                                add_special_tokens=True,
                                max_length=max_length,
                                padding='max_length',
                                truncation=False,
                                return_attention_mask=True,
                                return_tensors='pt')
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return {'input_ids': input_ids, 'attention_mask': attention_masks}
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_contexts_encodings = encode_plus(train_tc_pairs, tokenizer)
dev_contexts_encodings = encode_plus(dev_tc_pairs, tokenizer)
train_contexts_encodings['input_ids'].shape
X_train_context_input_ids = train_contexts_encodings['input_ids'].to(device)
X_train_context_attention_masks = train_contexts_encodings['attention_mask'].to(device)

X_dev_context_input_ids = dev_contexts_encodings['input_ids'].to(device)
X_dev_context_attention_masks = dev_contexts_encodings['attention_mask'].to(device)

X_train_context_input_ids.shape
# X_dev_context_input_ids.shape
train_labels = torch.tensor(train_tc_labels, dtype=torch.long).to(device)
dev_labels = torch.tensor(dev_tc_labels, dtype=torch.long).to(device)
train_tc_labels.shape
torch.save(X_train_context_input_ids, pickle_save_path + 'X_train_tc_input_ids.pt')
torch.save(X_train_context_attention_masks, pickle_save_path + 'X_train_tc_attention_masks.pt')
torch.save(train_labels, pickle_save_path + 'train_labels_tc.pt')

torch.save(X_dev_context_input_ids, pickle_save_path + 'X_dev_tc_input_ids.pt')
torch.save(X_dev_context_attention_masks, pickle_save_path + 'X_dev_tc_attention_masks.pt')
torch.save(dev_labels, pickle_save_path + 'dev_labels_tc.pt')
X_train_context_input_ids = torch.load(pickle_save_path + 'X_train_tc_input_ids.pt')
X_train_context_attention_masks = torch.load(pickle_save_path + 'X_train_tc_attention_masks.pt')
train_labels = torch.load(pickle_save_path + 'train_labels_tc.pt')

X_dev_context_input_ids = torch.load(pickle_save_path + 'X_dev_tc_input_ids.pt')
X_dev_context_attention_masks = torch.load(pickle_save_path + 'X_dev_tc_attention_masks.pt')
dev_labels = torch.load(pickle_save_path + 'dev_labels_tc.pt')
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=2)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()
train_contexts_encodings['input_ids'].shape
train_dataset = TensorDataset(X_train_context_input_ids, X_train_context_attention_masks, train_labels)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

dev_dataset = TensorDataset(X_dev_context_input_ids, X_dev_context_attention_masks, dev_labels)
dev_loader = DataLoader(dev_dataset, batch_size=16, shuffle=True)
def train_step(model, input_ids, attention_mask, labels):
    model.train()
    optimizer.zero_grad()
    predictions = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = predictions[0]
    loss.backward()
    optimizer.step()
    return loss.item()
wandb.init(
    # set the wandb project where this run will be logged
    project="NLP_Project",
    group='CRC',
    name='Bert-crc-tc-run1',

    config={
        "architecture": "bert",
        'context': 'target-utterance',
        'input shape': 'flattened',
        "epochs": 3,
        "learning_rate": 1e-5
    }
)
config = wandb.config

num_epochs = config.epochs   # Number of epochs, adjust as needed
train_losses = []
dev_losses = []


for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}')
    epoch_train_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids, attention_mask, labels = batch
        input_ids.to(device)
        attention_mask.to(device)
        labels.to(device)
        loss = train_step(model, input_ids, attention_mask, labels)
        epoch_train_loss += loss
    epoch_train_loss /= len(train_loader)
    train_losses.append(epoch_train_loss)
    
    # Validation
    model.eval()
    total_dev_loss = 0
    with torch.no_grad():
        for batch in dev_loader:
            input_ids, attention_mask, labels = batch
            input_ids.to(device)
            attention_mask.to(device)
            labels.to(device)
            predictions = model(input_ids, attention_mask=attention_mask, labels=labels)
            dev_loss = predictions[0]
            total_dev_loss += dev_loss.item()
    avg_dev_loss = total_dev_loss / len(dev_loader)
    dev_losses.append(avg_dev_loss)
    
    wandb.log({'train_loss': epoch_train_loss, 'val_loss': avg_dev_loss})
    print(f'Training Loss: {epoch_train_loss}, Validation Loss: {avg_dev_loss}')
model.save_pretrained(model_save_path + 'crc-bert-m1')
# tokenizer.save_pretrained(model_save_path)
plt.figure(figsize=(8, 4))
# plt.plot(range(1, num_epochs+1), train_losses, label='Training Loss')
# plt.plot(range(1, num_epochs+1), dev_losses, label='Validation Loss')
plt.plot(range(1, num_epochs), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs), dev_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in tqdm(dev_loader, desc=f"Eval Minibatch"):
        input_ids, attention_mask, labels = batch
        input_ids.to(device)
        attention_mask.to(device)
        labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions.append(np.argmax(outputs.logits.to('cpu').numpy(), axis=1))
        true_labels.extend(labels.cpu().numpy())
predictions = np.concatenate(predictions, axis=0)
np.array(true_labels)
(predictions == 1).any()
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')
precision = precision_score(true_labels, predictions, average='weighted')
recall = recall_score(true_labels, predictions, average='weighted')
print(classification_report(true_labels, predictions))
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')